Notes
- Tokenizer:set max_length = 512 because of RuntimeError: The size of tensor a (600) must match the size of tensor b (512) at non-single

In [1]:
# installing pytorch lightning and transformer
! pip install --quiet lightning
! pip install --quiet transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.2/802.2 kB 15.2 MB/s eta 0:00:00


In [2]:
# import libraries
import pandas as pd
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import plotly.express as px
from plotly.offline import init_notebook_mode
import re
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer
import spacy

tqdm.pandas()
spacy_eng = spacy.load("en_core_web_sm")
nltk.download('stopwords')
lemm = WordNetLemmatizer()
init_notebook_mode(connected=True)
sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = (20,8)
plt.rcParams['font.size'] = 18

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding, Layer, Dense, Dropout, MultiHeadAttention, LayerNormalization, Input, GlobalAveragePooling1D
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

# importing all necessary packages
from os import listdir
from os.path import join
from sklearn.model_selection import train_test_split
import string
from torch.utils.data import Dataset, DataLoader
import torch
import torchmetrics
import pytorch_lightning as pl
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
torch.manual_seed(42)
import numpy as np
np.random.seed(42)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
# import data from google drive
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/BT5151 + BT5153 /BT5153 - Fake Review Detection/fake reviews dataset.csv')
df.head(3)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...


In [5]:
# Tokenize the text
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Define a function for tokenization
def tokenize_text(batch):
    return tokenizer(batch, truncation=True, max_length=512, padding=True, return_tensors='pt')

# Train-test-split
sentences = df['text_'].values
label = df['label'].values
X_train, X_val, y_train, y_val = train_test_split(sentences, label, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=42)

# Tokenize text data
# Train-test-split
sentences_train = X_train.tolist()
sentences_val = X_val.tolist()
sentences_test = X_test.tolist()

# Tokenize text data
X_train_tokenized = tokenize_text(sentences_train)
X_val_tokenized = tokenize_text(sentences_val)
X_test_tokenized = tokenize_text(sentences_test)


# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

# Transform encoded Y data to tensor type
y_train_tensor = torch.tensor(y_train_encoded)
y_val_tensor = torch.tensor(y_val_encoded)
y_test_tensor = torch.tensor(y_test_encoded)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [6]:
import torch
from torch.utils.data import Dataset

class Setup(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.inputs['input_ids'])

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.inputs.items()}
        item['labels'] = self.labels[idx]
        return item

# CREATE AN ITERABLE DATASET FOR DATALOADERS
train_dataset = Setup(X_train_tokenized, y_train_tensor)
valid_dataset = Setup(X_val_tokenized,y_val_tensor)
test_dataset = Setup(X_test_tokenized, y_test_tensor)



# CREATE DATALOADERS
from torch.utils.data import DataLoader
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=5,
    shuffle=True,
    #num_workers=2
)

valid_loader = DataLoader(
    dataset=valid_dataset,
    batch_size=5,
    shuffle=False,
    #num_workers=2
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=5,
    shuffle=False,
    #num_workers=2
)
print(len(train_loader))
print(len(valid_loader))
print(len(test_loader))

6469
809
809


In [7]:
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torchmetrics import Precision, Recall, F1Score, AUROC
# write your code here

# GET THE PRETRAINTED MODEL AND SET #LABELS
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2)

# FREEZE ALL LAYES
for param in model.parameters():
  param.requires_grad = False

# UNFREEZE THE FINAL CLASSIFIER
#for param in model.pre_classifier.parameters():
  #param.requires_grad = True
for param in model.classifier.parameters():
  param.requires_grad = True

class LightningModel(pl.LightningModule):
  def __init__(self, model, learning_rate=2e-3):
      super().__init__()

      self.learning_rate = learning_rate
      self.model = model

      # Define metrics
      self.train_acc = torchmetrics.Accuracy(task="binary", num_classes=2)
      self.val_acc = torchmetrics.Accuracy(task="binary", num_classes=2)
      self.test_acc = torchmetrics.Accuracy(task="binary", num_classes=2)
      self.precision = Precision(task="binary", num_classes=2, average='binary')
      self.recall = Recall(task="binary", num_classes=2, average='binary')
      self.f1 = F1Score(task="binary", num_classes=2, average='binary')
      self.auroc = AUROC(task="binary", num_classes=2)

  def forward(self, input_ids, attention_mask, labels):
    return self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

  def training_step(self, batch, batch_idx):
      # forward pass
      outputs = self(batch["input_ids"], attention_mask=batch["attention_mask"],
                      labels=batch["labels"])
      # evaluate train loss
      self.log("train_loss", outputs["loss"])
      # evaluate train accuracy
      logits = outputs["logits"]
      predicted_labels = torch.argmax(logits, 1)
      self.train_acc(predicted_labels, batch["labels"])
      self.log("train_acc", self.train_acc, prog_bar=True)
      # compute and log precision, recall, and F1
      self.precision(predicted_labels, batch["labels"])
      self.recall(predicted_labels, batch["labels"])
      self.f1(predicted_labels, batch["labels"])
      self.log("train_precision", self.precision, prog_bar=True)
      self.log("train_recall", self.recall, prog_bar=True)
      self.log("train_f1", self.f1, prog_bar=True)
      return outputs["loss"]

  def validation_step(self, batch, batch_idx):
      # forward pass
      outputs = self(batch["input_ids"], attention_mask=batch["attention_mask"],
                       labels=batch["labels"])
      # evaluate valid loss
      self.log("val_loss", outputs["loss"], prog_bar=True)
      # evaluate valid accuracy
      logits = outputs["logits"]
      predicted_labels = torch.argmax(logits, 1)
      self.val_acc(predicted_labels, batch["labels"])
      self.log("val_acc", self.val_acc, prog_bar=True)
      # compute and log precision, recall, and F1
      self.precision(predicted_labels, batch["labels"])
      self.recall(predicted_labels, batch["labels"])
      self.f1(predicted_labels, batch["labels"])
      self.log("val_precision", self.precision, prog_bar=True)
      self.log("val_recall", self.recall, prog_bar=True)
      self.log("val_f1", self.f1, prog_bar=True)

  def test_step(self, batch, batch_idx):
      # forward pass
      outputs = self(batch["input_ids"], attention_mask=batch["attention_mask"],
                       labels=batch["labels"])
      self.log("test_loss", outputs['loss'])
      # evaluate test accuracy
      logits = outputs["logits"]
      predicted_labels = torch.argmax(logits, 1)
      self.test_acc(predicted_labels, batch["labels"])
      self.log("test_acc", self.test_acc, prog_bar=True)
      # compute and log precision, recall, and F1
      self.precision(predicted_labels, batch["labels"])
      self.recall(predicted_labels, batch["labels"])
      self.f1(predicted_labels, batch["labels"])
      self.log("test_precision", self.precision, prog_bar=True)
      self.log("test_recall", self.recall, prog_bar=True)
      self.log("test_f1", self.f1, prog_bar=True)


  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

lightning_model = LightningModel(model)

logger = TensorBoardLogger("distilbert/", name="finetuning", version="original")
callbacks = [
    ModelCheckpoint(
    dirpath= '/content/drive/MyDrive/BT5151 + BT5153 /BT5153 - Fake Review Detection',
    filename="best_model_distilbert",
    monitor="val_acc",
    mode="max",
    save_top_k=1
)
]


# FINE-TUNE THE MODEL
trainer = pl.Trainer(
    max_epochs=5,
    callbacks=callbacks, #accelerator="gpu", uncomment if gpu available
    logger=logger,
    log_every_n_steps=1,
)
trainer.fit(model=lightning_model,
            train_dataloaders=train_loader,
            val_dataloaders=valid_loader)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:653: Checkpoint directory /content/drive/MyDrive/BT5151 + BT5153 /BT5153 - Fake Review Detection exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbac

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [8]:
trainer.validate(dataloaders=valid_loader)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:145: `.validate(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.validate(ckpt_path='best')` to use the best model or `.validate(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/drive/MyDrive/BT5151 + BT5153 /BT5153 - Fake Review Detection/best_model_distilbert.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/drive/MyDrive/BT5151 + BT5153 /BT5153 - Fake Review Detection/best_model_distilbert.ckpt


Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │    0.8830076456069946     │
│          val_f1           │    0.8835262060165405     │
│         val_loss          │    0.27949732542037964    │
│       val_precision       │    0.8894397616386414     │
│        val_recall         │    0.8776907920837402     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 0.27949732542037964,
  'val_acc': 0.8830076456069946,
  'val_precision': 0.8894397616386414,
  'val_recall': 0.8776907920837402,
  'val_f1': 0.8835262060165405}]

In [9]:
# write your code here
trainer.test(lightning_model, dataloaders=test_loader, ckpt_path="best")

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/drive/MyDrive/BT5151 + BT5153 /BT5153 - Fake Review Detection/best_model_distilbert.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/drive/MyDrive/BT5151 + BT5153 /BT5153 - Fake Review Detection/best_model_distilbert.ckpt
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.8835311532020569     │
│          test_f1          │    0.8830394744873047     │
│         test_loss         │    0.28191447257995605    │
│      test_precision       │    0.8889999985694885     │
│        test_recall        │    0.8771583437919617     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.28191447257995605,
  'test_acc': 0.8835311532020569,
  'test_precision': 0.8889999985694885,
  'test_recall': 0.8771583437919617,
  'test_f1': 0.8830394744873047}]

Overfitting: XLNet has a large number of parameters, which makes it more prone to overfitting, especially when working with small datasets. Regularization techniques and careful hyperparameter tuning are necessary to mitigate overfitting and ensure generalization to unseen data.